In [8]:
import ipywidgets as widgets

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = "aaddcd39d3b94eff8d638832094ad11c"
CLIENT_SECRET = "744795edffb748188abdf154a5078ea0"

auth_manager = SpotifyClientCredentials(CLIENT_ID,CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
a = sp.search(q='The Beatles',type='artist',market='US')

artistID = a['artists']['items'][0]['id']
for item in a['artists']['items']:
    print(item['name'] + '\n')

The Beatles

The Beatles Complete On Ukulele

Tony Sheridan

The Beatles Recovered Band

beatles

The Beatles Revival Band

The Silver Beatles

The Tape-beatles

Barack Obama with The Beatles Complete On Ukulele Community Choir

The Next Beatless



In [ ]:
alb = sp.search(q="OK Human Weezer",type='album',market='US')
for item in alb['albums']['items']:
    print(item['name'] + ' by ' + item['artists'][0]['name'])

In [ ]:
track = sp.search(q="Dead Roses Weezer", type='track', market='US')
deadRoses = track['tracks']['items'][0]['id']
for item in track['tracks']['items']:
    print(item['name'] + ' from ' + item['album']['name'] + ' by ' + item['artists'][0]['name'])
    
track = sp.search(q="I Hate Everything About You", type='track',market='US')
iheay = track['tracks']['items'][0]['id']

track = sp.search(q="Donna The Lumineers",type='track',market='US')
donna = track['tracks']['items'][0]['id']

track = sp.search(q="Kiss the Rain Yiruma", type='track', market='US')
ktr = track['tracks']['items'][0]['id']

track = sp.search(q="I've Got a Name Jim Croce", type='track', market='US')
igan = track['tracks']['items'][0]['id']

In [ ]:
songs = [deadRoses,iheay,donna,ktr,igan]

af = sp.audio_features(songs)
for f in af:
    print('danceability:',f['danceability'])
    print('energy:',f['energy'])
    print('loudness:',f['loudness'])
    print('speechiness:',f['speechiness'])
    print('acousticness:',f['acousticness'])
    print('instrumentalness:',f['instrumentalness'])
    print('liveness:',f['liveness'])
    print('valence:',f['valence'])
    print('tempo:',f['tempo'])
    print('key:',f['key'])
    print('mode:',f['mode'])
    print()